## This is a tool to support talk map

In [1]:
import requests
import numpy as np
from pybtex.database.input import bibtex
from bs4 import BeautifulSoup as bs
import os
import glob

In [2]:
def convertAddressBing(address):
    # The bounds parameter defines the latitude/longitude coordinates
    # of the southwest and northeast corners of this bounding box
    API_KEY = "USE_YOUR_OWN_KEY"
    params = {
        'addressLine': address,
        'key': API_KEY,
        'strictMatch': 1,
    }
    baseurl = "http://dev.virtualearth.net/REST/v1/Locations?"
    res = requests.get(baseurl, params=params).json()
    if "errorDetails" in res.keys():
        print(res["errorDetails"])
        return np.NAN, np.NAN, np.NAN
    elif res['resourceSets'][0]['estimatedTotal'] == 0:
        return np.NAN, np.NAN, np.NAN
    else:
        lat, long = res['resourceSets'][0]['resources'][0]['point']['coordinates']
        found_address = res['resourceSets'][0]['resources'][0]["name"]
        return lat, long, found_address

def conference_info(cite_name):
    parser = bibtex.Parser()
    bibdata = parser.parse_file(cite_name)
    conference_obj = None
    for bib_id in bibdata.entries:
        b = bibdata.entries[bib_id].fields
        conference_obj = {
            "title": b["title"],
            "conference": b["booktitle"],
            "year": b["year"],
            "location": b["location"]
        }
        author_list = []
        for author in bibdata.entries[bib_id].persons["author"]:
            author_names = []
            author_info = [author.first_names, author.middle_names, author.last_names]
            for i in author_info:
                if len(i) > 0:
                    author_names.append(i[0])
            author_name = " ".join(author_names)
            author_list.append(author_name)
        conference_obj["authors"] = author_list
    return conference_obj

def generate_html_text(cite_name, match_name="Zongrun Li"):
    conference_obj = conference_info(cite_name)
    html_text = """
                <li>
                    <div class="conference-title">
                        %s
                    </div>
            """ % conference_obj["title"]

    for i in range(0, len(conference_obj["authors"])):
        author = conference_obj["authors"][i]
        if author == match_name:
            html_text += "<strong>" + match_name + "</strong>"
        else:
            html_text += author
        
        if i != len(conference_obj["authors"]) - 1:
            html_text += ", "
    
    html_text += """
        <br><em>%s</em>, %s
        <br>
    </li>
    """% (conference_obj["conference"], conference_obj["year"])
    
    soup = bs(html_text)
    html_text = soup.prettify()
    return html_text

In [3]:
# create JS
# ['Baltimore, MD, USA', 39.28870812, -76.63482396, '<div class="conference-title">Modeling the Regional Air Quality Impacts of Prescribed Burning at a Military Base in Southeastern United States</div><strong>Zongrun Li</strong>, <a>Yongtao Hu</a>, <a>M Talat Odman</a>, <a>Armistead G Russell</a><br><em>American Meteorological Society 103rd Annual Meeting</em>, 2024']
# process location and cluster locations
talks_folder_relative = "./talks"
sub_paths = glob.glob(os.path.join(talks_folder_relative, '*'))
locations = {}
for cur_path in sub_paths:
    location_name = conference_info(cur_path)["location"]
    lat, lon, bing_name = convertAddressBing(location_name)
    # locations.append((lon, lat))
    print("%s %s" % (location_name, bing_name))
    if (lon, lat) in locations.keys():
        locations[(lon, lat)]["file"].append(cur_path)
    else:
        locations[(lon, lat)] = {"file": [cur_path], "name": location_name}

Baltimore, MD Baltimore Ave, Baltimore, MD 21222
Denver, CO CO-2, Denver, CO 80246
Chapel Hill, NC Chapel Hill Creamery Rd, Chapel Hill, NC 27516
Chapel Hill, NC Chapel Hill Creamery Rd, Chapel Hill, NC 27516


In [4]:
# generate js variable
for location in locations.keys():
    cur_lon, cur_lat = location
    cur_info = locations[location]
    cur_list = [cur_info["name"], cur_lat, cur_lon]
    html_txt = ""
    for file in cur_info["file"]:
        cur_html = generate_html_text(file)
        html_txt += cur_html
    cur_list.append(html_txt)
    print(cur_list, end=", \n")

['Baltimore, MD', 39.258417, -76.5259275, '<li>\n <div class="conference-title">\n  Comparisons of High-Spatiotemporal Resolution Air Quality Modeling Systems for Simulating Prescribed Burning at Military Bases in the Southeastern United States\n </div>\n <strong>\n  Zongrun Li\n </strong>\n , M Talat Odman, Yongtao Hu, Susan O’Neill, Rime El, L Gregory Huey, David Tanner, Rodney J Weber, Armistead G Russell\n <br/>\n <em>\n  American Meteorological Society 104th Annual Meeting\n </em>\n , 2024\n <br/>\n</li>\n'], 
['Denver, CO', 39.6964363, -104.9405432, '<li>\n <div class="conference-title">\n  Modeling the Regional Air Quality Impacts of Prescribed Burning at a Military Base in Southeastern United States\n </div>\n <strong>\n  Zongrun Li\n </strong>\n , Yongtao Hu, M Talat Odman, Armistead G Russell\n <br/>\n <em>\n  American Meteorological Society 103rd Annual Meeting\n </em>\n , 2023\n <br/>\n</li>\n'], 
['Chapel Hill, NC', 35.93653646, -79.12412823, '<li>\n <div class="conference